In [30]:
import numpy as np
import random
import torch.nn as nn
import torch.optim as optim
import torch
import pandas as pd

In [31]:
data = pd.read_csv("Data\\preguntas.csv", sep=',', encoding = "ISO-8859-1")

In [32]:
X = np.load('Data\\X.npy', allow_pickle=True)
y = np.load('Data\\y.npy', allow_pickle=True)

In [33]:
#y_oh = np.zeros((y.shape[0], max(y)+1))
#y_oh[np.arange(y.shape[0]), y] = 1

In [34]:
random.seed(0)
np.random.seed(0)
idx = np.arange(X.shape[0])
random.shuffle(idx)  
btra = np.random.choice(idx, int(0.8*X.shape[0]), replace=False)
btes = [i for i in idx if i not in btra]

In [35]:
Xtra = X[btra]
ytra = y[btra]

Xtes = X[btes]
ytes = y[btes]

In [36]:
Xtes.shape

(202, 576)

In [8]:
def batch_generator(X, Y, n_batches):  
    
    random.seed(123)
    
    batch_size = X.shape[0] // n_batches
    
    idx = np.arange(X.shape[0])
    random.shuffle(idx)    
    idx = idx[:n_batches*batch_size]
        
    for i in range(n_batches):            
        bi = np.random.choice(idx, batch_size, replace=False)
        X_batch = X[bi]
        Y_batch = Y[bi]
        idx = [i for i in idx if i not in bi]
        yield (X_batch,Y_batch)       
        
        
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc11 = nn.Linear(576, 24, bias=True)
        self.fc12 = nn.Linear(24, 13, bias=True) 
                
    def forward(self, x):
        x1 = torch.tanh(self.fc11(x))
        x1 = self.fc12(x1)     
        return x1
    
    
def fit(X, Y, Xt, Yt, net, optimizer, error, n_epochs, n_batches, device, PATH, min_val_loss = float('inf')):
    
    net = net.to(device)    
    losses = []    
    val_losses = []

    val_inputs = torch.FloatTensor(Xt)
    val_labels = torch.tensor(Yt, dtype=torch.long)
    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device)  
    
    for epoch in range(n_epochs):  # loop over the dataset multiple times
    
        running_loss = 0  
         
        
        for batch_x, batch_y in batch_generator(X, Y, n_batches):  
            
            # zero the parameter gradients
            optimizer.zero_grad()
            
            # get the inputs
            inputs = torch.FloatTensor(batch_x)
            labels = torch.tensor(batch_y, dtype=torch.long)
            inputs, labels = inputs.to(device), labels.to(device)             
                
    
            # forward + backward + optimize
            outputs = net.forward(inputs)
            loss = error(outputs, labels)
                        
            loss.backward()    #obtain gradients      
            optimizer.step()   #optimize
                
            running_loss += loss.item()      
                
        running_loss = running_loss/n_batches    
        
        with torch.no_grad():
            val_outputs = net.forward(val_inputs)
            val_loss = error(val_outputs, val_labels) 
        
        losses.append(running_loss)   
        val_losses.append(val_loss.item())
        
        
        print('Epoch {0}: Training Loss: {1} Validation Loss: {2}'.format(epoch+1, running_loss, val_loss.item()))
        
        if val_loss.item() < min_val_loss:
            torch.save(net.state_dict(), PATH)
            print('New Checkpoint Saved into PATH')
            min_val_loss = val_loss.item()
        
        
def fweights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n-1)
        #y = 0.0001
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)        
        

In [9]:
n_epochs = 15000
lr = 0.001
n_batches = 3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
PATH = 'Data\\model_checkpoint.pt'

In [10]:
weights = np.sum(np.unique(ytes, return_counts=True)[1])/np.unique(ytes, return_counts=True)[1]
class_weights = torch.FloatTensor(weights).cuda()

In [11]:
net = Net1()
net.apply(fweights_init_normal)
criterion = nn.CrossEntropyLoss(weight=class_weights)
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9)

In [12]:
fit(Xtra, ytra, Xtes, ytes, net, optimizer, criterion, n_epochs, n_batches, device, PATH)

Epoch 1: Training Loss: 2.6303813457489014 Validation Loss: 2.617147922515869
New Checkpoint Saved into PATH
Epoch 2: Training Loss: 2.6187355518341064 Validation Loss: 2.6034090518951416
New Checkpoint Saved into PATH
Epoch 3: Training Loss: 2.598628362019857 Validation Loss: 2.5871829986572266
New Checkpoint Saved into PATH
Epoch 4: Training Loss: 2.5775736967722573 Validation Loss: 2.574556350708008
New Checkpoint Saved into PATH
Epoch 5: Training Loss: 2.5616602102915444 Validation Loss: 2.5686254501342773
New Checkpoint Saved into PATH
Epoch 6: Training Loss: 2.55218513806661 Validation Loss: 2.565044641494751
New Checkpoint Saved into PATH
Epoch 7: Training Loss: 2.546469767888387 Validation Loss: 2.5621063709259033
New Checkpoint Saved into PATH
Epoch 8: Training Loss: 2.541802088419596 Validation Loss: 2.5596210956573486
New Checkpoint Saved into PATH
Epoch 9: Training Loss: 2.5379830996195474 Validation Loss: 2.5571510791778564
New Checkpoint Saved into PATH
Epoch 10: Training

New Checkpoint Saved into PATH
Epoch 78: Training Loss: 2.4227632681528726 Validation Loss: 2.474470615386963
New Checkpoint Saved into PATH
Epoch 79: Training Loss: 2.4216310183207193 Validation Loss: 2.4734604358673096
New Checkpoint Saved into PATH
Epoch 80: Training Loss: 2.4199535846710205 Validation Loss: 2.4724295139312744
New Checkpoint Saved into PATH
Epoch 81: Training Loss: 2.417672554651896 Validation Loss: 2.4712893962860107
New Checkpoint Saved into PATH
Epoch 82: Training Loss: 2.417555014292399 Validation Loss: 2.470165967941284
New Checkpoint Saved into PATH
Epoch 83: Training Loss: 2.4156695206960044 Validation Loss: 2.4692115783691406
New Checkpoint Saved into PATH
Epoch 84: Training Loss: 2.4132705529530845 Validation Loss: 2.468064546585083
New Checkpoint Saved into PATH
Epoch 85: Training Loss: 2.4123123486836753 Validation Loss: 2.467118978500366
New Checkpoint Saved into PATH
Epoch 86: Training Loss: 2.4106053511301675 Validation Loss: 2.4661953449249268
New Che

Epoch 157: Training Loss: 2.3148081302642822 Validation Loss: 2.388615846633911
New Checkpoint Saved into PATH
Epoch 158: Training Loss: 2.3127084573109946 Validation Loss: 2.387582778930664
New Checkpoint Saved into PATH
Epoch 159: Training Loss: 2.311711072921753 Validation Loss: 2.386451244354248
New Checkpoint Saved into PATH
Epoch 160: Training Loss: 2.3092899322509766 Validation Loss: 2.3855361938476562
New Checkpoint Saved into PATH
Epoch 161: Training Loss: 2.3090540568033853 Validation Loss: 2.3843936920166016
New Checkpoint Saved into PATH
Epoch 162: Training Loss: 2.3069239457448325 Validation Loss: 2.3832709789276123
New Checkpoint Saved into PATH
Epoch 163: Training Loss: 2.3061808745066323 Validation Loss: 2.3822782039642334
New Checkpoint Saved into PATH
Epoch 164: Training Loss: 2.3044306437174478 Validation Loss: 2.3811872005462646
New Checkpoint Saved into PATH
Epoch 165: Training Loss: 2.303008794784546 Validation Loss: 2.379986047744751
New Checkpoint Saved into PAT

Epoch 234: Training Loss: 2.206247170766195 Validation Loss: 2.3013272285461426
New Checkpoint Saved into PATH
Epoch 235: Training Loss: 2.2051886717478433 Validation Loss: 2.300062894821167
New Checkpoint Saved into PATH
Epoch 236: Training Loss: 2.2034106254577637 Validation Loss: 2.2988884449005127
New Checkpoint Saved into PATH
Epoch 237: Training Loss: 2.20242969195048 Validation Loss: 2.2977964878082275
New Checkpoint Saved into PATH
Epoch 238: Training Loss: 2.200162649154663 Validation Loss: 2.296495199203491
New Checkpoint Saved into PATH
Epoch 239: Training Loss: 2.199256102244059 Validation Loss: 2.2952780723571777
New Checkpoint Saved into PATH
Epoch 240: Training Loss: 2.1983326276143393 Validation Loss: 2.293999433517456
New Checkpoint Saved into PATH
Epoch 241: Training Loss: 2.1962428092956543 Validation Loss: 2.2926225662231445
New Checkpoint Saved into PATH
Epoch 242: Training Loss: 2.1921302477518716 Validation Loss: 2.2914276123046875
New Checkpoint Saved into PATH


Epoch 314: Training Loss: 2.086108605066935 Validation Loss: 2.2043941020965576
New Checkpoint Saved into PATH
Epoch 315: Training Loss: 2.0848239262898765 Validation Loss: 2.2031989097595215
New Checkpoint Saved into PATH
Epoch 316: Training Loss: 2.084248701731364 Validation Loss: 2.2020344734191895
New Checkpoint Saved into PATH
Epoch 317: Training Loss: 2.082228342692057 Validation Loss: 2.2009875774383545
New Checkpoint Saved into PATH
Epoch 318: Training Loss: 2.080556790033976 Validation Loss: 2.1995391845703125
New Checkpoint Saved into PATH
Epoch 319: Training Loss: 2.07952610651652 Validation Loss: 2.198244333267212
New Checkpoint Saved into PATH
Epoch 320: Training Loss: 2.077337106068929 Validation Loss: 2.196901798248291
New Checkpoint Saved into PATH
Epoch 321: Training Loss: 2.076007604598999 Validation Loss: 2.195645809173584
New Checkpoint Saved into PATH
Epoch 322: Training Loss: 2.074878772099813 Validation Loss: 2.194072723388672
New Checkpoint Saved into PATH
Epoch

...

Epoch 14522: Training Loss: 0.08987815181414287 Validation Loss: 0.8298935890197754
Epoch 14523: Training Loss: 0.09029505401849747 Validation Loss: 0.8312031030654907
Epoch 14524: Training Loss: 0.08991867552200954 Validation Loss: 0.8313743472099304
Epoch 14525: Training Loss: 0.08991567542155583 Validation Loss: 0.8310025334358215
Epoch 14526: Training Loss: 0.08993158489465714 Validation Loss: 0.8297515511512756
Epoch 14527: Training Loss: 0.08985071629285812 Validation Loss: 0.8286849856376648
Epoch 14528: Training Loss: 0.09008740882078807 Validation Loss: 0.8284350037574768
Epoch 14529: Training Loss: 0.09010942528645198 Validation Loss: 0.8288527131080627
Epoch 14530: Training Loss: 0.08969485263029735 Validation Loss: 0.8290420770645142
Epoch 14531: Training Loss: 0.0899741401274999 Validation Loss: 0.830662727355957
Epoch 14532: Training Loss: 0.09000901877880096 Validation Loss: 0.831555187702179
Epoch 14533: Training Loss: 0.09011254211266835 Validation Loss: 0.831465363502

Epoch 14629: Training Loss: 0.08963559816281001 Validation Loss: 0.8303394913673401
Epoch 14630: Training Loss: 0.08963357657194138 Validation Loss: 0.8306695222854614
Epoch 14631: Training Loss: 0.08982285112142563 Validation Loss: 0.8318015336990356
Epoch 14632: Training Loss: 0.0903268630305926 Validation Loss: 0.8316090703010559
Epoch 14633: Training Loss: 0.08970436453819275 Validation Loss: 0.8317116498947144
Epoch 14634: Training Loss: 0.08967841416597366 Validation Loss: 0.83141028881073
Epoch 14635: Training Loss: 0.08997642497221629 Validation Loss: 0.8308570384979248
Epoch 14636: Training Loss: 0.0896116445461909 Validation Loss: 0.832191526889801
Epoch 14637: Training Loss: 0.08960949381192525 Validation Loss: 0.8321974277496338
Epoch 14638: Training Loss: 0.08960464845101039 Validation Loss: 0.8323137760162354
Epoch 14639: Training Loss: 0.08987535287936528 Validation Loss: 0.8308390378952026
Epoch 14640: Training Loss: 0.08974230786164601 Validation Loss: 0.83147752285003

Epoch 14734: Training Loss: 0.0893575797478358 Validation Loss: 0.8331401944160461
Epoch 14735: Training Loss: 0.08976914236942928 Validation Loss: 0.8327558040618896
Epoch 14736: Training Loss: 0.08999638259410858 Validation Loss: 0.8323557376861572
Epoch 14737: Training Loss: 0.08955218146244685 Validation Loss: 0.8320317268371582
Epoch 14738: Training Loss: 0.08990253259738286 Validation Loss: 0.8322010040283203
Epoch 14739: Training Loss: 0.08968377113342285 Validation Loss: 0.832105815410614
Epoch 14740: Training Loss: 0.08948531498511632 Validation Loss: 0.8318750858306885
Epoch 14741: Training Loss: 0.0892203797896703 Validation Loss: 0.8317580223083496
Epoch 14742: Training Loss: 0.08945487687985103 Validation Loss: 0.8314082622528076
Epoch 14743: Training Loss: 0.08964800586303075 Validation Loss: 0.8300338983535767
Epoch 14744: Training Loss: 0.0894281913836797 Validation Loss: 0.8317018747329712
Epoch 14745: Training Loss: 0.08947357287009557 Validation Loss: 0.8343051671981

Epoch 14831: Training Loss: 0.08902627974748611 Validation Loss: 0.8322609663009644
Epoch 14832: Training Loss: 0.0890636146068573 Validation Loss: 0.8328208923339844
Epoch 14833: Training Loss: 0.0897890105843544 Validation Loss: 0.8322517275810242
Epoch 14834: Training Loss: 0.08910136918226878 Validation Loss: 0.8336231112480164
Epoch 14835: Training Loss: 0.08904412885506947 Validation Loss: 0.8347533345222473
Epoch 14836: Training Loss: 0.08910524348417918 Validation Loss: 0.8345372080802917
Epoch 14837: Training Loss: 0.08932321270306905 Validation Loss: 0.8344631195068359
Epoch 14838: Training Loss: 0.0899152581890424 Validation Loss: 0.833129346370697
Epoch 14839: Training Loss: 0.09020260473092397 Validation Loss: 0.8340454697608948
Epoch 14840: Training Loss: 0.0891299123565356 Validation Loss: 0.8339709043502808
Epoch 14841: Training Loss: 0.08899734417597453 Validation Loss: 0.8327585458755493
Epoch 14842: Training Loss: 0.08901405334472656 Validation Loss: 0.83293056488037

Epoch 14938: Training Loss: 0.08965527266263962 Validation Loss: 0.8352470993995667
Epoch 14939: Training Loss: 0.08860837419827779 Validation Loss: 0.8348225951194763
Epoch 14940: Training Loss: 0.08945334578553836 Validation Loss: 0.8353123664855957
Epoch 14941: Training Loss: 0.08895560105641682 Validation Loss: 0.8338255286216736
Epoch 14942: Training Loss: 0.08884250621000926 Validation Loss: 0.8350280523300171
Epoch 14943: Training Loss: 0.08888552834590276 Validation Loss: 0.8351559042930603
Epoch 14944: Training Loss: 0.08921308318773906 Validation Loss: 0.8340942859649658
Epoch 14945: Training Loss: 0.08895651499430339 Validation Loss: 0.8337408304214478
Epoch 14946: Training Loss: 0.0887955551346143 Validation Loss: 0.833669126033783
Epoch 14947: Training Loss: 0.0886365423599879 Validation Loss: 0.8345111608505249
Epoch 14948: Training Loss: 0.08853031943241756 Validation Loss: 0.8343045711517334
Epoch 14949: Training Loss: 0.08879065265258153 Validation Loss: 0.834992289543

In [37]:
PATH = 'Data\\model_checkpoint.pt'
device = torch.device('cpu')
net = Net1()
net.load_state_dict(torch.load(PATH, map_location = device))

In [38]:
error = nn.CrossEntropyLoss()

In [39]:
with torch.no_grad():
    net.eval()
    inputs = torch.FloatTensor(Xtes)
    labels = torch.tensor(ytes, dtype=torch.long)
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net.forward(inputs)
    loss = error(outputs, labels) 
    print(loss.item())

0.815489649772644


In [40]:
param_dict = {}
for v,p in enumerate(net.parameters()):
    param_dict[v] = p.data.numpy()
    
W0 = param_dict[0].T
b0 = param_dict[1]
W1 = param_dict[2].T
b1 = param_dict[3]


h0 = np.matmul(Xtes, W0) + b0
h1 = np.tanh(h0)
h2 = np.matmul(h1, W1) + b1
h3 = np.exp(h2)
o = h3/np.sum(h3,axis=1).reshape(-1,1)

In [41]:
o.shape

(202, 13)

In [42]:
np.save('Data\\param_dict.npy', param_dict)

In [43]:
def y_hat(prob, topics):    
    return topics[np.argmax(prob)]
    

In [44]:
pred_labels = []
topics = ['Jubilacion Patronal', 'Consultoria', 'Renuncia/Despido/Desahucio', 'IESS', 
                 'Greeting', 'Contacto', 'No Topic', 'Queja', 'Otros servicios', 'Charlas/Capacitaciones', 
                      'Hi Five', 'job seeker', 'Facturacion/Retencion/Cobros']
for c in o:
    pred_labels.append(y_hat(c, topics))

In [45]:
preguntas = data.Pregunta[btes]
labels = data.Tema[btes]
ambig = data.ambiguedad[btes]

In [46]:
Xtes = np.column_stack((preguntas, labels, pred_labels, ambig, Xtes, o))
#Xtes = np.column_stack((preguntas, labels, pred_labels, Xtes, o))

In [47]:
data2 = pd.DataFrame(Xtes)


In [48]:
vocab = np.load('Data\\vocab.npy', allow_pickle=True)
colnames = np.concatenate((['pregunta', 'label', 'pred_label', 'ambig'], vocab, ['n_token', 'perc_greet', 'capacitacion', 'polarity'], ['Jubilacion Patronal', 'Consultoria', \
                                                                                                     'Renuncia/Despido/Desahucio', 'IESS', 'Greeting', \
                                                                                                         'Contacto', 'No Topic', 'Queja', 'Otros servicios', \
                                                                                                 'Charlas/Capacitaciones', 'Hi Five', 'job seeker', 'Facturacion/Retencion/Cobros']))
data2.columns = colnames


In [49]:
data2.head(3)


,pregunta,label,pred_label,ambig,buen,dia,mi,nombr,llam,empres,...,IESS,Greeting,Contacto,No Topic,Queja,Otros servicios,Charlas/Capacitaciones,Hi Five,job seeker,Facturacion/Retencion/Cobros
0,Hola qué tal,Greeting,Greeting,NaN,0,0,0,0,0,0,...,0.0196924,0.575278,0.0117529,0.0418878,0.00780163,0.110079,0.0266009,0.0171291,0.0969172,0.00848771
1,Hola buenos días como calcular un finiquito,Renuncia/Despido/Desahucio,Renuncia/Despido/Desahucio,NaN,1,1,0,0,0,0,...,0.00467586,0.0397423,0.000463164,6.33128e-06,7.4005e-05,0.000176887,0.000293868,8.49566e-05,0.00118814,0.000725514
2,Buenas tardes mi consulta es para mi padre que...,Jubilacion Patronal,IESS,ambivalente,1,0,0,0,0,0,...,0.939896,0.000385766,0.000399942,0.000635039,4.90935e-05,0.000385507,0.00126023,4.56116e-05,6.7998e-05,7.69816e-05


In [50]:
data2.to_csv('Data\\Xtes.csv', sep=',')